In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-20o6xsvc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-20o6xsvc
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=b2b64c796b11442a787156ca8a62ed97fa326dd385ded523563a0126105aad75
  Stored in directory: /tmp/pip-ephem-wheel-cache-8bpe6948/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [10]:
from PIL import Image
import pickle
import torch
import skimage.io as io
import clip
import os
import numpy as np
import pandas as pd
from PIL import Image
import re
import spacy

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
#captions_file = "/content/drive/MyDrive/ImageCaptioning/captions_curated.csv"
base_path = "/content/drive/MyDrive/Open_Source_Projects/NewTryImageCaptionPrediction/Flicker8k"

# Caption Directories
captions_file = base_path + "/captions.csv"
token_file = base_path + "/Extracts/tokens.csv"

# Image Directories
image_zip_path = base_path + "/Images.zip"
image_dir='30k'
imaze_unzip_path = "/content/"+ image_dir
!mkdir image_dir
data_folder = "/content/drive/MyDrive/Open_Source_Projects/Image_Captioning"
image_data_location = os.path.join(data_folder, "Flickr8k_Dataset/Flicker8k_Dataset")
train_image_csv =  base_path + "/Extracts/train_images.txt"
test_image_csv = base_path + "/Extracts/test_images.txt"
val_image_csv = base_path + "/Extracts/val_images.txt"
image_embedding_file = base_path + "/Extracts/clip_image_embeddings.pkl"

mkdir: cannot create directory ‘image_dir’: File exists


In [11]:
captionsDF = pd.read_csv(captions_file)
captionsDF['comment'] = captionsDF['comment'].apply(lambda sentence: re.sub('[^a-zA-Z ]+', '', sentence.lower()))
sentence_list = captionsDF['comment'].tolist()
spacy_eng = spacy.load('en_core_web_sm')
itos = {0:"<PAD>",1:"<SOS>",2:"<EOS>",3:"<UNK>"}
words = []
for sentence in sentence_list:
  for word in [token.text.lower() for token in spacy_eng.tokenizer(sentence)]:
    words.append(word)
tokens = ["<PAD>","<SOS>","<EOS>","<UNK>"]
for word in set(words):
  tokens.append(word)
tdf = pd.DataFrame({"Tokens":tokens})
tdf["Index"] = tdf.index
tdf.to_csv(token_file)

In [ ]:
from zipfile import ZipFile
with ZipFile(image_zip_path, 'r') as zObject:
  zObject.extractall(path=imaze_unzip_path)
zObject.close()

In [ ]:
def write2file(filename, content):
  file1 = open(filename, 'w')
  for i in content:
    file1.write(i+'\n')
  file1.close()

In [ ]:
test_size = 0.2
val_size = 0.1
images = os.listdir(image_data_location)
random.shuffle(images)
write2file(test_image_csv, images[:int(test_size*len(images))])
write2file(val_image_csv, images[int(test_size*len(images)):int((test_size+val_size)*len(images))])
write2file(train_image_csv, images[int((test_size+val_size)*len(images)):])

In [14]:
images = os.listdir(image_data_location)

In [15]:
image_embeddings = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model, preprocess = clip.load('ViT-B/32', device=device, jit=False)
for img_name in images:
  image = io.imread(image_data_location+"/"+img_name)
  image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
  with torch.no_grad():
    prefix = clip_model.encode_image(image).cpu()
    image_embeddings[img_name] = prefix

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 52.7MiB/s]


In [16]:
image_embedding_file = base_path + "/Extracts/clip_image_embeddings.pkl"
with open(image_embedding_file, 'wb') as f:
    pickle.dump(image_embeddings, f)


In [ ]:
with open(image_embedding_file, 'rb') as f:
    all_data = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [ ]:
all_data['1000092795.jpg']

tensor([[ 1.8225e-01,  3.5156e-01,  1.8616e-02, -1.1377e-01,  3.4058e-02,
          1.3025e-01,  2.1655e-01,  1.3940e-01,  5.3314e-02, -2.6343e-01,
         -5.1270e-02, -1.9666e-01,  1.2842e-01,  2.8107e-02,  5.6061e-02,
          9.6191e-02, -8.1689e-01,  3.2544e-01, -6.7505e-02,  2.6978e-01,
          3.1128e-01,  1.0199e-01, -4.3152e-02, -2.5879e-01,  6.9519e-02,
         -4.0308e-01,  4.8767e-02,  2.4939e-01,  2.5635e-01, -9.3750e-02,
         -3.2495e-01, -1.7249e-01,  3.4302e-01,  4.2358e-02,  9.8096e-01,
          2.9883e-01, -1.6907e-02, -8.6182e-02,  4.5288e-01, -5.2930e-01,
         -3.6896e-02,  1.4185e-01, -6.4453e-02, -2.6367e-01,  3.5498e-01,
         -1.2871e+00,  1.8665e-01,  1.4549e-02,  3.9429e-02,  6.3525e-01,
         -3.6377e-01,  2.9419e-02, -6.7078e-02, -3.0371e-01,  4.0503e-01,
          6.3049e-02, -2.1790e-01,  9.4788e-02, -6.4209e-01, -1.0931e-01,
          2.7771e-02,  4.9652e-02,  1.6492e-01, -7.1228e-02,  1.9312e-01,
         -1.4294e-01,  3.1525e-02,  2.